In [0]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    numpy-1.14.2               |   py36hdbf6ddf_0         4.0 MB
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    pandas-0.23.4              |   py36hf8a1672_0        27.8 MB  conda-forge
    altair-2.2.2               |        py36_1001         494 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        32.4 MB

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py36_1001       conda-forge
    branca:  0.3.1-py_0           

<a id='item1'></a>

## 1. Download and Explore Dataset

In [0]:
!wget -q -O 'chicago_data.json' https://data.cityofchicago.org/api/views/unjd-c2ca/rows.json
print('Data downloaded!')

Data downloaded!


#### Load and explore the data

In [0]:
import pandas as pd
df = pd.read_csv("chicago_nbhd.csv")

In [0]:
df.head()

,PRI_NEIGH,the_geom,SEC_NEIGH,SHAPE_AREA,SHAPE_LEN
0,Grand Boulevard,MULTIPOLYGON (((-87.60670812560372 41.81681377...,BRONZEVILLE,4.849250e+07,28196.837157
1,Printers Row,MULTIPOLYGON (((-87.62760697485348 41.87437097...,PRINTERS ROW,2.162138e+06,6864.247156
2,United Center,MULTIPOLYGON (((-87.66706868914602 41.88885187...,UNITED CENTER,3.252051e+07,23101.363745
3,Sheffield & DePaul,MULTIPOLYGON (((-87.65833494805533 41.92166144...,SHEFFIELD & DEPAUL,1.048259e+07,13227.049745
4,Humboldt Park,MULTIPOLYGON (((-87.74059567509266 41.88782316...,HUMBOLDT PARK,1.250104e+08,46126.751351


#### Tranform the data into a *pandas* dataframe

In [0]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.

In [0]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [0]:
num = len(df)

Then let's loop through the data and fill the dataframe one row at a time.

In [0]:
for i in range(num):
    borough = neighborhood_name = df['PRI_NEIGH'][i]
    
    if "&" in borough:
        borough = borough.split("&")[0].strip(" ")
    neighborhood_name = df['SEC_NEIGH'][i]
    
    address = '{}, Chicago, IL'.format(borough)
    
    geolocator = Nominatim()
    location = geolocator.geocode(address)
    
    if location is not None:
        neighborhood_lat = location.latitude
        neighborhood_lon = location.longitude

        neighborhoods = neighborhoods.append({'Borough': borough,
                                              'Neighborhood': neighborhood_name,
                                              'Latitude': neighborhood_lat,
                                              'Longitude': neighborhood_lon}, ignore_index=True)

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


Quickly examine the resulting dataframe.

And make sure that the dataset has all 97 boroughs and 194 neighborhoods.

In [0]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 97 boroughs and 194 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Chicago.

In [0]:
address = 'Chicago, IL'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chicago are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Chicago are 41.8755616, -87.6244212.


#### Create a map of Chicago with neighborhoods superimposed on top.

In [0]:
# create map of New York using latitude and longitude values
map_chicago = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chicago)  
    
map_chicago

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

In [0]:
gb_data = neighborhoods
gb_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Grand Boulevard,BRONZEVILLE,41.813923,-87.617272
1,Printers Row,PRINTERS ROW,41.873787,-87.628900
2,United Center,UNITED CENTER,41.880655,-87.674164
3,Sheffield,SHEFFIELD & DEPAUL,41.934459,-87.653925
4,Humboldt Park,HUMBOLDT PARK,41.905767,-87.704174


Let's get the geographical coordinates of Chicago.



let's visualize Chicago with the neighborhoods in it.

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [0]:
CLIENT_ID = '3R25XLUYK5YX3TUZPQPPUGDVST2LWR2HBRT3MLV1MT0VNBLL' # your Foursquare ID
CLIENT_SECRET =  '4WVDEYHTT2EJRK4JZVEMJC2T1SC1UOFPB0XH1NAMWCHOS3IR' # your Foursquare Secret
VERSION = '20180605'  # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3R25XLUYK5YX3TUZPQPPUGDVST2LWR2HBRT3MLV1MT0VNBLL
CLIENT_SECRET:4WVDEYHTT2EJRK4JZVEMJC2T1SC1UOFPB0XH1NAMWCHOS3IR


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [0]:
gb_data.loc[1, 'Neighborhood']

'PRINTERS ROW'

Get the neighborhood's latitude and longitude values.

In [0]:
neighborhood_latitude = gb_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = gb_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name =gb_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of BRONZEVILLE are 41.8139226, -87.6172724.


In [0]:
address = 'Chicago, IL'

geolocator = Nominatim()
location = geolocator.geocode(address)
neighborhood_name = 'Chicago'
neighborhood_latitude = location.latitude
neighborhood_longitude = location.longitude
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


Latitude and longitude values of Chicago are 41.8755616, -87.6244212.


#### Now, let's get the top 100 venues that are in the first neighborhood within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [0]:
# type your answer here


LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude , VERSION, radius, LIMIT)



Double-click __here__ for the solution.
<!-- The correct answer is:
LIMIT = 100 # limit of number of venues returned by Foursquare API
-->

<!--
radius = 500 # define radius
-->

<!--
\\ # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
--> 

Send the GET request and examine the resutls

In [0]:
results = requests.get(url).json()
results.keys()

dict_keys(['meta', 'response'])

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [0]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [0]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Auditorium Theatre,Theater,41.876058,-87.625303
1,Cafecito,Cuban Restaurant,41.875724,-87.626386
2,Hostelling International Chicago,Hostel,41.875757,-87.626537
3,Osaka Sushi Express & Fresh Fruit Smoothies,Sushi Restaurant,41.876969,-87.624534
4,Dunkin' Donuts,Donut Shop,41.876771,-87.624423


And how many venues were returned by Foursquare?

In [0]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


<a id='item2'></a>

## 2. Explore Neighborhoods in Chicago

#### Let's create a function to repeat the same process to all the neighborhoods in Chicago

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
        except:
            continue
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *Chicago_venues*.

In [0]:
# type your answer here
gb_venues = getNearbyVenues(names=gb_data['Neighborhood'],
                                   latitudes=gb_data['Latitude'],
                                   longitudes=gb_data['Longitude']
                                  )

BRONZEVILLE
PRINTERS ROW
UNITED CENTER
SHEFFIELD & DEPAUL
HUMBOLDT PARK
GARFIELD PARK
NORTH LAWNDALE
LITTLE VILLAGE
ARMOUR SQUARE,CHINATOWN
AVALON PARK,CALUMET HEIGHTS
CHATHAM,BURNSIDE
BELMONT CRAIGIN,HERMOSA
IRVING PARK,AVONDALE
LOGAN SQUARE
AVALON PARK,CALUMET HEIGHTS
SOUTHEAST SIDE
WEST PULLMAN
MIDWAY AIRPORT
BACK OF THE YARDS
ENGLEWOOD
SOUTH SHORE, GRAND CROSSING
ASHBURN
MOUNT GREENWOOD,MORGAN PARK
MOUNT GREENWOOD,MORGAN PARK
OHARE
JACKSON PARK
LOOP
PULLMAN
RIVERDALE
SOUTHEAST SIDE
GREEKTOWN
BRONZEVILLE
MUSEUM CAMPUS
EDGEWATER
LAKE VIEW
LINCOLN PARK
STREETERVILLE
LINCOLN SQUARE
OAKLAND,KENWOOD
GRANT PARK
WEST LOOP
BACK OF THE YARDS
ANDERSONVILLE
WOODLAWN
PORTAGE PARK
RUSH & DIVISION
KENWOOD,OAKLAND
ROGERS PARK
JEFFERSON PARK
SAUGANASH,FOREST GLEN
NORTH PARK,ALBANY PARK
NORTH PARK,ALBANY PARK
IRVING PARK,AVONDALE
DUNNING
WEST RIDGE
UPTOWN
NORWOOD PARK
STREETERVILLE
SOUTH SHORE, GRAND CROSSING
CHATHAM,BURNSIDE
SOUTH CHICAGO
WASHINGTON HEIGHTS,ROSELAND
NORTH CENTER
SOUTHEAST SIDE
WASH

Double-click __here__ for the solution.
<!-- The correct answer is:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )
--> 

#### Let's check the size of the resulting dataframe

In [0]:
print(gb_venues.shape)
gb_venues.head()

(5679, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,BRONZEVILLE,41.813923,-87.617272,Parkway Ballroom,41.813142,-87.616064,Food
1,BRONZEVILLE,41.813923,-87.617272,Ain't She Sweet Cafe,41.816817,-87.613004,Coffee Shop
2,BRONZEVILLE,41.813923,-87.617272,Peach's Restaurant,41.809481,-87.617009,Breakfast Spot
3,BRONZEVILLE,41.813923,-87.617272,Sip & Savor,41.816817,-87.612876,Coffee Shop
4,BRONZEVILLE,41.813923,-87.617272,Blanc,41.813585,-87.616003,Art Gallery


Let's check how many venues were returned for each neighborhood

In [0]:
gb_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
ANDERSONVILLE,168,168,168,168,168,168
"ARCHER HEIGHTS,WEST ELSDON",72,72,72,72,72,72
"ARMOUR SQUARE,CHINATOWN",120,120,120,120,120,120
ASHBURN,8,8,8,8,8,8
AUBURN GRESHAM,18,18,18,18,18,18
AUSTIN,24,24,24,24,24,24
"AVALON PARK,CALUMET HEIGHTS",30,30,30,30,30,30
BACK OF THE YARDS,42,42,42,42,42,42
"BELMONT CRAGIN,HERMOSA",32,32,32,32,32,32


#### Let's find out how many unique categories can be curated from all the returned venues

In [0]:
print('There are {} uniques categories.'.format(len(gb_venues['Venue Category'].unique())))

There are 297 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [0]:
# one hot encoding
gb_onehot = pd.get_dummies(gb_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
gb_onehot['Neighborhood'] = gb_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [gb_onehot.columns[-1]] + list(gb_onehot.columns[:-1])
gb_onehot = gb_onehot[fixed_columns]

gb_onehot.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,African Restaurant,Airport Lounge,Airport Terminal,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,New American Restaurant,Nightclub,Non-Profit,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Road,Rock Club,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Track,Trail,Train,Train Station,Turkish Restaurant,Ukrainian Restaurant,Vege

And let's examine the new dataframe size.

In [0]:
gb_onehot.shape

(5679, 298)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [0]:
gb_grouped = gb_onehot.groupby('Neighborhood').mean().reset_index()
gb_grouped

,Neighborhood,ATM,Accessories Store,Adult Boutique,African Restaurant,Airport Lounge,Airport Terminal,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,New American Restaurant,Nightclub,Non-Profit,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Road,Rock Club,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Track,Trail,Train,Train Station,Turkish Restaurant,Ukrainian Restaurant,Vege

#### Let's confirm the new size

In [0]:
gb_grouped.shape

(76, 298)

#### Let's print each neighborhood along with the top 5 most common venues

In [0]:
num_top_venues = 5

for hood in gb_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp =gb_grouped[gb_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ANDERSONVILLE----
                    venue  freq
0                  Lounge  0.04
1      Italian Restaurant  0.04
2  Furniture / Home Store  0.04
3               Pet Store  0.04
4             Coffee Shop  0.04


----ARCHER HEIGHTS,WEST ELSDON----
                  venue  freq
0    Mexican Restaurant  0.19
1     Mobile Phone Shop  0.08
2  Fast Food Restaurant  0.08
3                   Bar  0.08
4         Grocery Store  0.08


----ARMOUR SQUARE,CHINATOWN----
                venue  freq
0  Chinese Restaurant  0.37
1    Asian Restaurant  0.08
2        Dessert Shop  0.05
3              Bakery  0.05
4   Korean Restaurant  0.05


----ASHBURN----
                venue  freq
0  Light Rail Station  0.25
1      Cosmetics Shop  0.25
2         Pizza Place  0.25
3  Italian Restaurant  0.25
4                 ATM  0.00


----AUBURN GRESHAM----
                  venue  freq
0  Fast Food Restaurant  0.22
1              Pharmacy  0.11
2             BBQ Joint  0.11
3        Cosmetics Shop  0.11
4     

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [0]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = gb_grouped['Neighborhood']

for ind in np.arange(gb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(gb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ANDERSONVILLE,Pet Store,Italian Restaurant,Sandwich Place,Furniture / Home Store,Coffee Shop,Lounge,Breakfast Spot,Salon / Barbershop,Pizza Place,New American Restaurant
1,"ARCHER HEIGHTS,WEST ELSDON",Mexican Restaurant,Bar,Mobile Phone Shop,Fast Food Restaurant,Grocery Store,Gym / Fitness Center,Video Store,Park,Flea Market,Bank
2,"ARMOUR SQUARE,CHINATOWN",Chinese Restaurant,Asian Restaurant,Dessert Shop,Korean Restaurant,Bakery,Bubble Tea Shop,Tea Room,Dim Sum Restaurant,Sculpture Garden,Sports Bar
3,ASHBURN,Italian Restaurant,Pizza Place,Cosmetics Shop,Light Rail Station,Zoo,Eastern European Restaurant,Doctor's Office,Dog Run,Donut Shop,Drugstore
4,AUBURN GRESHAM,Fast Food Restaurant,Greek Restaurant,Discount Store,Cosmetics Shop,American Restaurant,Pharmacy,BBQ Joint,Lounge,Doctor's Office,Dog Run
5,AUSTIN,Park,Bus Station,Grocery Store,Liquor Store,Wings Joint,Mobile Phone Shop,Shoe Repair,Train Station,Gym,Donut Shop
6,"AVALON PARK,CALUMET HEIGHTS",Fast Food Restaurant,Burger Joint,Grocery Store,Pizza Place,Diner,Cajun / Creole Restaurant,Sandwich Place,Bus Station,Breakfast Spot,Boutique
7,BACK OF THE YARDS,Fast Food Restaurant,American Restaurant,Chinese Restaurant,Spanish Restaurant,Park,Pizza Place,Sporting Goods Shop,Brewery,Light Rail Station,Sandwich Place
8,"BELMONT CRAGIN,HERMOSA",Mexican Restaurant,Grocery Store,Bakery,Nightclub,Discount Store,Laundromat,Department Store,Burger Joint,Chinese Restaurant,Thrift / Vintage Store
9,"BELMONT CRAIGIN,HERMOSA",Diner,Supermarket,Optical Shop,Latin American Restaurant,Department Store,Park,Food Truck,Check Cashing Service,Discount Store,Recording Studio


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [0]:
# set number of clusters
kclusters = 5

gb_grouped_clustering = gb_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(gb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 1, 4, 4, 4, 0, 4, 4, 1, 4], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [0]:
# add clustering labels
neighborhoods_venues_sorted['Cluster Labels'] = kmeans.labels_

In [0]:
# merge data to add latitude/longitude for each neighborhood
gb_merged = gb_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

gb_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Grand Boulevard,BRONZEVILLE,41.813923,-87.617272,Fast Food Restaurant,Coffee Shop,Gym / Fitness Center,Cosmetics Shop,Art Gallery,Clothing Store,Park,Sandwich Place,Breakfast Spot,Shipping Store,4.0
1,Printers Row,PRINTERS ROW,41.873787,-87.628900,Pizza Place,Coffee Shop,Sandwich Place,Gym / Fitness Center,Hotel,Bookstore,Gym,American Restaurant,Boutique,Indian Restaurant,4.0
2,United Center,UNITED CENTER,41.880655,-87.674164,Outdoor Sculpture,Stadium,Hockey Arena,Sports Bar,Mediterranean Restaurant,Brazilian Restaurant,Bar,Basketball Stadium,Fried Chicken Joint,Park,4.0
3,Sheffield,SHEFFIELD & DEPAUL,41.934459,-87.653925,Bar,Sports Bar,Thai Restaurant,Pizza Place,Pub,Salon / Barbershop,Japanese Restaurant,Spa,Lounge,Seafood Restaurant,4.0
4,Humboldt Park,HUMBOLDT PARK,41.905767,-87.704174,Park,Grocery Store,Café,Museum,Food Truck,Baseball Field,Lake,Zoo,Doctor's Office,Dog Run,0.0


Finally, let's visualize the resulting clusters

In [0]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(gb_merged['Latitude'], gb_merged['Longitude'], gb_merged['Neighborhood'],gb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    try:
        cluster= int(cluster)
    except:
        continue
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [0]:
gb_merged.loc[gb_merged['Cluster Labels'] == 0,gb_merged.columns[[1] + list(range(5, gb_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
4,HUMBOLDT PARK,Grocery Store,Café,Museum,Food Truck,Baseball Field,Lake,Zoo,Doctor's Office,Dog Run,0.0
6,NORTH LAWNDALE,Convenience Store,Park,Zoo,Eastern European Restaurant,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,0.0
10,"CHATHAM,BURNSIDE",Intersection,Bus Station,Home Service,Donut Shop,Train Station,Fast Food Restaurant,Cuban Restaurant,Cupcake Shop,Farmers Market,0.0
22,"MOUNT GREENWOOD,MORGAN PARK",Grocery Store,Cosmetics Shop,Scenic Lookout,Track,Train Station,Vineyard,BBQ Joint,Convenience Store,Ethiopian Restaurant,0.0
23,"MOUNT GREENWOOD,MORGAN PARK",Grocery Store,Cosmetics Shop,Scenic Lookout,Track,Train Station,Vineyard,BBQ Joint,Convenience Store,Ethiopian Restaurant,0.0
25,JACKSON PARK,Dog Run,Golf Driving Range,Garden,Track,Sculpture Garden,Other Great Outdoors,Dry Cleaner,Drugstore,Donut Shop,0.0
27,PULLMAN,Sandwich Place,Food,Furniture / Home Store,Café,Train Station,National Park,Grocery Store,Museum,Dry Cleaner,0.0
38,"OAKLAND,KENWOOD",Public Art,Track,Boutique,Lake,Discount Store,Drugstore,Dive Bar,Doctor's Office,Dog Run,0.0
43,WOODLAWN,Bus Station,Coffee Shop,Park,Dry Cleaner,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,0.0
49,"SAUGANASH,FOREST GLEN",Fast Food Restaurant,Asian Restaurant,Intersection,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,0.0


#### Cluster 2

In [0]:
gb_merged.loc[gb_merged['Cluster Labels'] == 1, gb_merged.columns[[1] + list(range(5, gb_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
47,ROGERS PARK,Bakery,Convenience Store,Chinese Restaurant,Train Station,Donut Shop,Bar,Sushi Restaurant,Farmers Market,Breakfast Spot,1.0
60,SOUTH CHICAGO,Mexican Restaurant,Bar,Harbor / Marina,Electronics Store,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,1.0
68,"MARQUETTE PARK,GAGE PARK",Pizza Place,Convenience Store,Bakery,American Restaurant,Fast Food Restaurant,Intersection,Home Service,Asian Restaurant,Sandwich Place,1.0
75,LOWER WEST SIDE,Pizza Place,Bakery,Thrift / Vintage Store,Plaza,Coffee Shop,Discount Store,Latin American Restaurant,Breakfast Spot,Bar,1.0
77,"ARCHER HEIGHTS,WEST ELSDON",Bar,Mobile Phone Shop,Fast Food Restaurant,Grocery Store,Gym / Fitness Center,Video Store,Park,Flea Market,Bank,1.0
78,"BRIGHTON PARK,MCKINLEY PARK",Grocery Store,Pizza Place,Burger Joint,Taco Place,Diner,Chinese Restaurant,Seafood Restaurant,Big Box Store,Sandwich Place,1.0
79,"BRIGHTON PARK,MCKINLEY PARK",Grocery Store,Pizza Place,Burger Joint,Taco Place,Diner,Chinese Restaurant,Seafood Restaurant,Big Box Store,Sandwich Place,1.0
83,"ARCHER HEIGHTS,WEST ELSDON",Bar,Mobile Phone Shop,Fast Food Restaurant,Grocery Store,Gym / Fitness Center,Video Store,Park,Flea Market,Bank,1.0
84,"MARQUETTE PARK,GAGE PARK",Pizza Place,Convenience Store,Bakery,American Restaurant,Fast Food Restaurant,Intersection,Home Service,Asian Restaurant,Sandwich Place,1.0
91,OLD TOWN,BBQ Joint,Liquor Store,Southern / Soul Food Restaurant,Mexican Restaurant,Convenience Store,Doctor's Office,Dog Run,Donut Shop,Drugstore,1.0


#### Cluster 3

In [0]:
gb_merged.loc[gb_merged['Cluster Labels'] == 2,gb_merged.columns[[1] + list(range(5, gb_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
34,LAKE VIEW,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Eastern European Restaurant,Electronics Store,2.0
131,LAKE VIEW,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Eastern European Restaurant,Electronics Store,2.0


#### Cluster 4

In [0]:
gb_merged.loc[gb_merged['Cluster Labels'] == 3, gb_merged.columns[[1] + list(range(5, gb_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
56,NORWOOD PARK,Campground,Zoo,Dry Cleaner,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,3.0
153,NORWOOD PARK,Campground,Zoo,Dry Cleaner,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,3.0


#### Cluster 5

In [0]:
gb_merged.loc[gb_merged['Cluster Labels'] == 4, gb_merged.columns[[1] + list(range(5, gb_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,BRONZEVILLE,Coffee Shop,Gym / Fitness Center,Cosmetics Shop,Art Gallery,Clothing Store,Park,Sandwich Place,Breakfast Spot,Shipping Store,4.0
1,PRINTERS ROW,Coffee Shop,Sandwich Place,Gym / Fitness Center,Hotel,Bookstore,Gym,American Restaurant,Boutique,Indian Restaurant,4.0
2,UNITED CENTER,Stadium,Hockey Arena,Sports Bar,Mediterranean Restaurant,Brazilian Restaurant,Bar,Basketball Stadium,Fried Chicken Joint,Park,4.0
3,SHEFFIELD & DEPAUL,Sports Bar,Thai Restaurant,Pizza Place,Pub,Salon / Barbershop,Japanese Restaurant,Spa,Lounge,Seafood Restaurant,4.0
5,GARFIELD PARK,Intersection,Recreation Center,Pharmacy,Trail,Park,Garden,Train Station,Zoo,Discount Store,4.0
7,LITTLE VILLAGE,Coffee Shop,Art Museum,Bookstore,Museum,Bakery,Hotel,Music Venue,Theater,Dance Studio,4.0
8,"ARMOUR SQUARE,CHINATOWN",Asian Restaurant,Dessert Shop,Korean Restaurant,Bakery,Bubble Tea Shop,Tea Room,Dim Sum Restaurant,Sculpture Garden,Sports Bar,4.0
9,"AVALON PARK,CALUMET HEIGHTS",Burger Joint,Grocery Store,Pizza Place,Diner,Cajun / Creole Restaurant,Sandwich Place,Bus Station,Breakfast Spot,Boutique,4.0
11,"BELMONT CRAIGIN,HERMOSA",Supermarket,Optical Shop,Latin American Restaurant,Department Store,Park,Food Truck,Check Cashing Service,Discount Store,Recording Studio,4.0
12,"IRVING PARK,AVONDALE",Gym,Chinese Restaurant,Sandwich Place,Bus Line,Diner,Brewery,Donut Shop,Bus Station,Food Truck,4.0




The skeleton of this notebook was created by [Alex Aklson](https://www.linkedin.com/in/aklson/). 